In [10]:
import numpy
import random
from collections import defaultdict
import urllib
import math
import random
import collections
import string

In [11]:
def parse(path):
  file = open(path, 'r')
  for l in file:
    yield eval(l)

print("Reading data...")
products = list(parse("/Users/Silvia/Downloads/reviews.json"))
# http://jmcauley.ucsd.edu/data/amazon/qa/qa_Tools_and_Home_Improvement.json.gz
qa = list(parse("/Users/Silvia/Downloads/qa.json"))
print("done")

Reading data...
done


In [12]:
reviews = defaultdict(lambda: [])
for product in products:
    reviews[product["asin"]].append(product["reviewText"])

In [13]:
def tf(term, document):
    count = collections.defaultdict(int)
    for word in document.split():
        count[word] += 1

    return count[term]/len(document.split())

In [14]:
#docDict is the union of a question and the set of all reviews for said question.
def idf(term, docSet):
    count = 0
    for doc in docSet:
        if term in doc.lower():
            count += 1
    return math.log(len(docSet)/(count+1))

In [15]:
def feature(document, docSet):
    document = document.lower()
    document = ''.join([c for c in document if not (c in string.punctuation)])
    
    feat = collections.defaultdict(int)
    for term in document.split():
        feat[term] = tf(term, document) * idf(term, docSet)
    return feat

In [16]:
# queryFeat is a feature vector for the query and reviewFeat is the feature vector for the review
def cosineSimilarity(queryFeat, reviewFeat):
    # Find the words the 2 dictionaries have in common
    querySet = set(queryFeat.keys())
    reviewSet = set(reviewFeat.keys())
    allWords = querySet.union(reviewSet)
    
    # Find the cosine similarity
    numerator = 0
    mag1 = 0
    mag2 = 0
    for word in allWords:
        numerator = numerator + queryFeat[word] * reviewFeat[word]
        mag1 = mag1 + queryFeat[word]**2
        mag2 = mag2 + reviewFeat[word]**2
    if mag1 > 0 and mag2 > 0:
        return (numerator/((mag1*mag2)**0.5))
    else:
        return -1

In [22]:
questionSet = set()


def randomQuestion(qa, reviews):
    #numReviews = 0
    for question in qa:
    #while numReviews < 70:
        
        question = random.choice(qa)
        if (question['questionType'] != "yes/no"): continue
        if (question['answerType'] == '?'): continue
        #if (question['asin'] in questionSet): continue
        questionText = question['question']
        asin = question['asin']
        answer = question['answerType']
        numReviews = len(reviews[asin])
        if (numReviews < 70): continue
    
        # reviewsSet is list of all review text for this product
        reviewsSet = reviews[asin]
        # docDict is union of questionText and reviewsDoc
        docSet = list(reviewsSet)
        docSet.append(questionText)

        # Make dictionaries for the query and review
        queryFeat = feature(questionText, docSet)
        cosSimList = {review : cosineSimilarity(queryFeat, feature(review, docSet)) for review in reviewsSet}
        cosSimList = sorted(cosSimList.items(), key=lambda x:x[1])

        reviewSample = [review[0] for review in cosSimList[:3]]
        reviewSample = reviewSample + [review[0] for review in cosSimList[-7:]]

        #print(questionText)
        #print(cosSimList[-3:])

        print(question['answerType'])


        yield [asin, questionText, reviewSample, answer]

In [23]:

with open("/Users/Silvia/Downloads/out.csv", 'w') as the_file:
    data = list(randomQuestion(qa, reviews))
    
    #for i in range(0,100):
    for d in data:
        asin = d[0]
        question = d[1]
        reviewSample = d[2]
        answer = d[3]
        questionSet.add(asin)
        for review in reviewSample:
            the_file.write(asin)
            the_file.write(',')
            the_file.write(question.replace(',', ''))
            the_file.write(',')
            the_file.write(review.replace(',', ''))
            the_file.write(',')
            the_file.write(answer.replace(',', ''))
            the_file.write('\n')

Y
Y
Y
N
Y
N
N
Y
Y
N
Y
N
Y
N
Y
Y
Y
Y
Y
Y
N
Y
Y
Y
N
Y
N
Y
Y
Y
Y
Y
Y
Y
Y
N
Y
Y
Y
N
Y
N
N
Y
Y
Y
N
N
Y
Y
N
N
Y
Y
N
Y
Y
Y
N
Y
Y
Y
Y
N
Y
Y
Y
Y
Y
Y
N
Y
N
Y
Y
Y
Y
Y
N
N
Y
Y
Y
Y
Y
Y
N
Y
Y
Y
N
Y
N
Y
Y
N
Y
N
Y
Y
Y
Y
Y
N
Y
Y
Y
N
Y
N
Y
N
Y
N
Y
Y
N
Y
Y
N
Y
Y
N
Y
N
Y
Y
N
Y
N
N
Y
Y
Y
Y
Y
Y
N
N
N
N
Y
N
Y
Y
Y
N
Y
Y
Y
N
Y
N
N
Y
Y
Y
N
N
N
Y
N
Y
N
N
N
Y
Y
Y
N
Y
Y
Y
N
Y
Y
N
Y
Y
Y
Y
Y
N
Y
N
Y
Y
N
Y
Y
N
N
Y
Y
Y
Y
Y
Y
N
Y
Y
N
Y
N
Y
N
Y
Y
N
Y
Y
Y
Y
Y
